<div style="text-align: right">
    <i>
        Alëna Aksënova <br>
        SCiL FLT workshop <br>
        Sunday tutorial
    </i>
</div>
 

# [_SigmaPie_](https://github.com/alenaks/SigmaPie) for subregular and subsequential grammar induction


This toolkit implements several **subregular language classes** (strictly local, tier-based strictly local, multiple tier-based strictly local, and strictly piecewise). For every class, it provides a set of parameters that can be specified (locality window, alphabet, set of restrictions, polarity of the grammar, etc.) and functions that can be applied to members of those classes (evaluate a word, extract a grammar from data, construct the corresponding finite state automata, and others). These models can be employed to encode **well-formedness conditions**.

Another type of finite models -- **transducers** -- are capable of capturing mappings, or **processes** rewriting the input sequence according to the encoded set of rules. Transduction learning algorithms extract the mapping from the observed input-output pairs. For example, OSTIA (Onward Subsequential Transducer Inference Algorithm) by [Oncina et. al (1993)](https://pdfs.semanticscholar.org/9058/01c8e75daacb27d70ccc3c0b587411b6d213.pdf) is a transduction learner that generalizes the training sample into a machine reading the input string symbol by symbol and outputting at every step the longest part of the output sequence known up to that moment.

Finally, I demonstrate the functionality of _SigmaPie_ using the simplified datasets exhibiting harmonic and dissimilation patterns.

## Accessing _SigmaPie_ toolkit

#### Way 1: running from the terminal
  1. Download the code from the [SigmaPie GitHub folder](https://github.com/alenaks/SigmaPie);
  2. Open the terminal and use `cd` to move to the `SigmaPie/code/` repository.
  3. Run Python3 compiler by typing `python3`.
  4. `from main import *` will load all the modules of the package.
 
  <img src="images/terminal.png" width="750">
  
#### Way 2: running from the Jupyter notebooks
  1. Download the code from the [SigmaPie GitHub folder](https://github.com/alenaks/SigmaPie).
  2. Modify the second line in the cell below so that it contains the correct path to `SigmaPie/code/`.
  3. Run that cell.

In [1]:
%cd ex_generator
from ex_generator import *

%cd
%cd SigmaPie/code/
from main import *

/home/alenaks/SigmaPie/tutorial/ex_generator
/home/alenaks
/home/alenaks/SigmaPie/code

You successfully loaded SigmaPie. 

Formal language classes and grammars available:
	* strictly piecewise: SP(alphabet, grammar, k, data, polar);
	* strictly local: SL(alphabet, grammar, k, data, edges, polar);
	* tier-based strictly local: TSL(alphabet, grammar, k, data, edges, polar, tier);
	* multiple tier-based strictly local: MTSL(alphabet, grammar, k, data, edges, polar).

Alternatively, you can initialize a transducer: FST(states, sigma, gamma, initial, transitions, stout).
Learning algorithm:
	OSTIA: ostia(sample, sigma, gamma).


## Languages, grammars and well-formedness conditions

Well-formedness conditions can be represented via a set of strings that follow some rule restricting the **shape of those strings**.
_Formal language theory_ explains how potentially infinite string sets, or _formal languages,_
can be generalized to grammars encoding the desired patterns and what properties those
grammars have.
It also allows one to compare different grammars regarding parameters such as **expressivity**.


**The Chomsky hierarchy** aligns the main classes of formal languages with respect to their expressive power [(Chomsky 1959)](http://www.cs.utexas.edu/~cannata/pl/Class%20Notes/Chomsky_1959%20On%20Certain%20Formal%20Properties%20of%20Grammars.pdf).

  * **Regular** grammars are as powerful as finite-state devices or regular expressions, and they cannot produce patterns that require counting up to an arbitrary number (no $a^{n}b^{n}$ patterns);
  * **Context-free** grammars have access to a potentially infinite _stack_ that allows them to produce patterns with nested dependencies, such as center embedding;
  * **Mildly context-sensitive** grammars are powerful enough to handle cross-serial dependencies such as some types of copying;
  * **Context-sensitive** grammars can handle unbounded copy languages and non-linear patterns such as $a^{2^{n}}$ for $n > 0$;
  * **Recursively enumerable** grammars are as powerful as any theoretically possible computer and generate languages such as $a^n$, where $n \in \textrm{primes}$.



<img src="images/chomhier.png" width="600">


Both phonology and morphology frequently display properties of regular languages.

**Phonology** does not require the power of center-embedding, which is a property of context-free languages. For example, consider a harmony where the first vowel agrees with the last vowel, the second vowel agrees with the pre-last one, etc. The following example shows this rule using English orthography.
    
    GOOD: "arugula", "tropicalization", "electrotelethermometer", etc.
    BAD:  any other word violating the rule.


While it is a theoretically possible pattern, harmonies of that type are unattested in natural languages.

**Morphology** avoids center-embedding as well. In [Aksënova et al. (2016)](https://www.aclweb.org/anthology/W16-2019) we show that it is possible to iterate prefixes with the meaning "after" in Russian. In Ilocano, where the same semantics are expressed via a circumfix, its iteration is prohibited.
    
    RUSSIAN: "zavtra" (tomorrow), "posle-zavtra" (the day after tomorrow), 
             "posle-posle-zavtra" (the day after the day after tomorrow), ...
    ILOCANO: "bigat" (morning), "ka-bigat-an" (the next morning),
             <*>"ka-ka-bigat-an-an" (the morning after the next one).

### Subregular language classes


Typological review of patterns shows that **phonology and morphology do not require the full power of regular languages**. As an example of an unattested pattern, [Heinz (2011)](http://jeffreyheinz.net/papers/Heinz-2011-CPF.pdf) provides a language where a word must have an even number of nasals to be well-formed. Regular languages can be sub-divided into another nested hierarchy of languages decreasing in their expressive power: **subregular hierarchy**.
Among some of the most important characteristics of subregular languages is their learnability only from positive data: more powerful classes require negative input as well.


<img src="images/subreg.png" width="250">


The _SigmaPie_ toolkit currently contains functionality for the following subregular language and grammar classes:
  * strictly piecewise (SP);
  * strictly local (SL);
  * tier-based strictly local (TSL);
  * multiple tier-based strictly local (MTSL).
  
| Language | Dependencies it can handle                                    |
|----------|---------------------------------------------------------------|
| SL       | _only_ local dependencies                                     |
| SP       | _only_ multiple long-distance dependencies _without_ blocking |
| TSL      | long-distance dependencies _with_ blocking                    |
| MTSL     | multiple long-distance dependencies _with_ blocking           |



**Negative strictly piecewise (`SP`)** grammars prohibit the occurrence of sequences of symbols at an arbitrary distance from each other. Every SP grammar is associated with the value of $k$ that defines the size of the longest sequence that this grammar can prohibit. Alternatively, if the grammar is positive, it lists all subsequences that are allowed in well-formed words of the language. **SP grammars capture only long-distance dependencies that do not include blockers.**

    k = 2
    POLARITY: negative
    GRAMMAR:  ab, ba
    LANGUAGE: accaacc, cbccc, cccacaaaa, ...
              <*>accacba, <*>bcccacbb, <*>bccccccca, ...

**Negative strictly $k$-local (`SL`)** grammars prohibit the occurrence of consecutive substrings consisting of up to $k$ symbols. The value of $k$ defines the longest substring that cannot be present in a well-formed string of a language. Positive SL grammars define substrings that can be present in the language.
To define _first_ and _last_ elements, SL languages use delimiters (">" and "<") that indicate the beginning and the end of the string. In phonology, changes involve adjacent segments very frequently, and the notion of locality is therefore extremely important. A discussion of local processes in phonology can be found in [(Chandlee 2014)](http://dspace.udel.edu/bitstream/handle/19716/13374/2014_Chandlee_Jane_PhD.pdf). **SL grammars only capture local dependencies.**

    k = 2
    POLARITY: positive
    GRAMMAR:  >a, ab, ba, b<
    LANGUAGE: ab, abab, abababab, ...
              <*>babab, <*>abaab, <*>bababba, ...
              
**Negative tier-based strictly local (`TSL`)** grammars operate just like strictly local ones, but they have the power to _ignore_ a certain set of symbols completely. The set of symbols that are not ignored are called **tier** symbols, and the ones that do not matter for the well-formedness of strings are the **non-tier** ones [(Heinz et al. 2011)](https://pdfs.semanticscholar.org/b934/bfcc962f65e19ae139426668e8f8054e5616.pdf). The representation of a string with all non-tier symbols ignored is a _tier image_ of that string, and then the TSL grammars can be defined as _SL grammars that operate over a tier._ **TSL grammars capture a single long-distance dependency that can possibly include blockers.**

    k = 2
    POLARITY: negative
    TIER:     b
    GRAMMAR:  ><, bb
    LANGUAGE: aaaabaaaa, b, aaaab, baaaa, aaabaaaa, ...
              <*>aaaaa, <*>aaaabaabaa, <*>baaabaaa, ...

**Negative multiple tier-based strictly local (`MTSL`)** grammars are a conjunction of multiple TSL grammars: they consist of several tiers, and a set of restrictions is defined for every one of those tiers. In fact, there are numerous examples from the typological literature showing that there are phonological patterns of complexity which are beyond the power of TSL languages. One example could be any pattern where several long-distance dependencies affect different sets of elements, see [McMullin (2016)](https://www.dropbox.com/s/txmk4efif9f5bvb/McMullin_Dissertation_UBC.pdf?dl=0) and [Aksënova and Deshmukh (2018)](https://www.aclweb.org/anthology/W18-0307.pdf) for examples and discussions of those patterns. **MTSL grammars capture multiple long-distance dependencies that can possibly include blockers.**

    k = 2
    POLARITY:   negative
    TIER_1:     a, o
    GRAMMAR_1:  oa, ao
    TIER_2:     p, b
    GRAMMAR_2:  bp, pb
    LANGUAGE:   obobo, apappp, opoppop, baaaaa, ...
                <*>ppapboo, <*>oobbbab, <*>poobbap, ...

The work here is based on **string representations**. The exemplified learning algorithms focus on **structural properties**, and are limited to non-probabilistic algorithms evaluating the well-formedness of input stings. This approach is currently extended to features ([Chandlee et al. 2019](https://www.aclweb.org/anthology/W19-5708/)) and autosegmental representations ([Chandlee and Jardine 2019](https://www.aclweb.org/anthology/Q19-1010/); [Rawski and Dolatian (to appear)](https://drive.google.com/file/d/19Ft6j7ta71uTTw3qkLRa6bqhR98caJGk/view)) in order to be more coherent with the representations used in linguistics. However, the statistical algorithms and the algorithms working with non-string-based representations are not implemented yet.

### Vowel harmony and consonant harmony

#### Pattern description

In Bukusu, vowels agree in height, and a liquid "l" assimilates to "r" if followed by "r" somewhere further in the word [(Odden 1994)](https://www.jstor.org/stable/415830?seq=1#metadata_info_tab_contents).

  * <b>r</b><i>ee</i>b-<i>e</i><b>r</b>- _ask-APPL_
  * <b>l</b><i>i</i>m-<i>i</i><b>l</b>- _cultivate-APPL_
  * <b>r</b><i>u</i>m-<i>i</i><b>r</b>- _send-APPL_
  
This pattern involves two long-distance assimilations: one of them affects vowels, and the other one is concerned with the consonants. To capture the big picture, we can simplify the dependency as follows: the two harmonic classes of vowels are mapped to "a" and "o", and the affected consonants are mapped to "b" and "p".

    Good strings: aaabbabba, oppopooo, aapapapp, obooboboboobbb, ...
    Bad strings:  <*>aabaoob, <*>paabab, <*>obabooo, ...
    Generalization: if a string contains "a", it cannot contain "o", and vice versa;
                    if a string contains "p", it cannot contain "b", and vice versa.
                    
<img src="images/mtsl.png" width="530">

In [2]:
harm_data = ['aabbaabb', 'abab', 'aabbab', 'abaabb', 'aabaab', 'abbabb', 'ooppoopp',
             'opop', 'ooppop', 'opoopp', 'oopoop', 'oppopp', 'aappaapp', 'apap',
             'aappap', 'apaapp', 'aapaap', 'appapp', 'oobboobb', 'obob', 'oobbob',
             'oboobb', 'ooboob', 'obbobb', 'aabb', 'ab', 'aab', 'abb', 'oopp', 'op',
             'oop', 'opp', 'oobb', 'ob', 'oob', 'obb', 'aapp', 'ap', 'aap', 'app',
             'aaa', 'ooo', 'bbb', 'ppp', 'a', 'o', 'b', 'p', '']

#### Initializing the SL, TSL, MTSL and SP classes

This pattern in indeed an instance of an MTSL grammar, however, we can try to learn it using other grammars as well. `help` can be used to see what parameters and methods which subregular class implements.

In [3]:
help(MTSL)

Help on class MTSL in module mtsl_class:

class MTSL(tsl_class.TSL)
 |  A class for tier-based strictly local grammars and languages.
 |  Attributes:
 |      alphabet (list): alphabet used in the language;
 |      grammar (list): the list of substructures;
 |      k (int): locality window;
 |      data (list): input data;
 |      edges (list): start- and end-symbols for the grammar;
 |      polar ("p" or "n"): polarity of the grammar;
 |      fsm (FSMFamily): a list of finite state machines that 
 |          corresponds to the grammar;
 |      tier (list): list of tuples, where every tuple lists elements
 |          of some tier.
 |  Learning for k > 2 is not implemented: requires more theoretical work.
 |  
 |  Method resolution order:
 |      MTSL
 |      tsl_class.TSL
 |      sl_class.SL
 |      grammar.L
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, alphabet=None, grammar=None, k=2, data=None, edges=['>', '<'], polar='p')
 |      Initializes the TSL

The first step is to initialize $4$ subregular grammars for every one of the available classes and provide the training sample.

In [4]:
sp_h = SP()
sl_h = SL()
tsl_h = TSL()
mtsl_h = MTSL()

sp_h.data = harm_data
sl_h.data = harm_data
tsl_h.data = harm_data
mtsl_h.data = harm_data

We can then simply fill the `L.alphabet` attributes by applying the method `L.extract_alphabet()`.

In [5]:
sp_h.extract_alphabet()
sl_h.extract_alphabet()
tsl_h.extract_alphabet()
mtsl_h.extract_alphabet()

print("SP alphabet:", sp_h.alphabet)
print("SL alphabet:", sl_h.alphabet)
print("TSL alphabet:", tsl_h.alphabet)
print("MTSL alphabet:", mtsl_h.alphabet)

SP alphabet: ['a', 'b', 'o', 'p']
SL alphabet: ['a', 'b', 'o', 'p']
TSL alphabet: ['a', 'b', 'o', 'p']
MTSL alphabet: ['a', 'b', 'o', 'p']


#### Learning the grammars

After the data and alphabet are established, we can extract the grammar with its corresponding complexity from every one of those classes using the method `L.learn()`.

In [6]:
sp_h.learn()
sl_h.learn()
tsl_h.learn()
mtsl_h.learn()

The learned **SP grammar** lists all possible $2$-long subsequences observed in the training sample. The sequences are represented as tuples instead of strings in order to avoid restricting the basic alphabet units to a single symbol.

In [7]:
print("Positive SP grammar:", sp_h.grammar)

Positive SP grammar: [('a', 'b'), ('b', 'a'), ('b', 'b'), ('a', 'a'), ('o', 'p'), ('p', 'o'), ('p', 'p'), ('o', 'o'), ('a', 'p'), ('p', 'a'), ('b', 'o'), ('o', 'b')]


In order to express the same generalization using a negative grammar, we can `L.switch_polarity()` of the grammar.

In [8]:
sp_h.switch_polarity()
print("Polarity of the SP grammar:", sp_h.check_polarity())
print("SP grammar:", sp_h.grammar)

Polarity of the SP grammar: n
SP grammar: [('a', 'o'), ('b', 'p'), ('o', 'a'), ('p', 'b')]


The negative **SL grammar** contains the same set of restrictions as its SP counterpart. So, the same set of restrictions is detected, even though SP grammars express long-distance restrictions, whereas SL grammars only limit local relations.

In [9]:
sl_h.switch_polarity()
print("Polarity of the SL grammar:", sl_h.check_polarity())
print("SL grammar:", sl_h.grammar)

Polarity of the SL grammar: n
SL grammar: [('a', 'o'), ('b', 'p'), ('o', 'a'), ('p', 'b')]


**TSL grammars** also express the same limitations over a tier as the previous two, and the tier includes all elements that participate in some sort of long-distance agreement. In this case, the tier includes all elements of the alphabet.

In [10]:
print("TSL tier:", tsl_h.tier)
tsl_h.switch_polarity()
print("Polarity of the TSL grammar:", tsl_h.check_polarity())
print("TSL grammar:", tsl_h.grammar)

TSL tier: ['a', 'b', 'o', 'p']
Polarity of the TSL grammar: n
TSL grammar: [('a', 'o'), ('b', 'p'), ('o', 'a'), ('p', 'b')]


In **MTSL grammars**, the value of the attribute `L.grammar` is represented in the following way:

    G = {
            tier_1 (tuple): tier_1_restrictions (list),
            tier_2 (tuple): tier_2_restrictions (list),
                ...
            tier_n (tuple): tier_n_restrictions (list)
        }
        
The learned grammar detected two tiers: a tier of vowels `("a", "o")` and a tier of consonants `("p", "b")`. For every one of these tiers, it learned the corresponding set of restrictions.

In [11]:
print("MTSL tiers:", mtsl_h.tier)
mtsl_h.switch_polarity()
print("Polarity of the MTSL grammar:", mtsl_h.check_polarity())
print("MTSL grammar:", mtsl_h.grammar)

MTSL tiers: [('b', 'p'), ('a', 'o')]
Polarity of the MTSL grammar: n
MTSL grammar: {('b', 'p'): [('b', 'p'), ('p', 'b')], ('a', 'o'): [('a', 'o'), ('o', 'a')]}


#### Generating samples

Now, when all the grammars are learned, we can generate new data using the `L.generate_sample(n, repeat, safe)` method.

The **SP-generated** data is consistent with the desired pattern: no "a" is followed by "o" in the generated strings of the language, and the consonantal agreement is preserved as well. SP grammar succeeded in capturing the pattern!

In [12]:
print(sp_h.generate_sample(25, repeat=False, safe=True))

['', 'bbbo', 'papp', 'pp', 'bb', 'a', 'p', 'apappappa', 'obbbobbobbb', 'bbob', 'ooboooo', 'app', 'ppoppp', 'oob', 'ooboboobooo', 'obo', 'bobobbb', 'b', 'boobboo', 'o', 'baba', 'bbabbbaa', 'booo', 'obbbbb', 'ppoo']


As we can see below, **SL grammar** captured the local effect of the learned pattern ("p" is never adjacent to "b", "o" is never adjacent to "a", etc.), but it failed to generalize it to long-distance relations.

In [13]:
print(sl_h.generate_sample(25, repeat=False))

['', 'pa', 'pap', 'oo', 'ab', 'aapooobobap', 'a', 'p', 'baaab', 'ppob', 'bapooo', 'op', 'obbobopooobbboobo', 'ppppopap', 'aboppapp', 'po', 'appp', 'babbabopp', 'abobbapppaapob', 'opapap', 'appaabbb', 'pab', 'b', 'o', 'oopo']


Similarly to the errors made by the SL grammar, **TSL grammar** only captured a local dependency, and failed to generalize the pattern.

In [14]:
print(tsl_h.generate_sample(25, repeat=False))

['', 'ooopappp', 'oopaabb', 'pp', 'a', 'p', 'boboob', 'ppap', 'op', 'aa', 'aaapa', 'boopapapa', 'po', 'appa', 'app', 'obbabo', 'papoop', 'ooobo', 'bapopopobb', 'ob', 'obo', 'baapppo', 'abb', 'b', 'aapa']


Increasing the locality window `L.k` did not help solve the issue: violations still occur, the only difference is that now they occur at a further distance away from each other.

In [15]:
tsl_h.k = 3
tsl_h.learn()
print(tsl_h.generate_sample(25, repeat=False))

['', 'aab', 'ppappp', 'pp', 'bb', 'abbb', 'a', 'p', 'appoob', 'oboboo', 'aa', 'aaaaa', 'ppaa', 'bboobb', 'opoobbabbbbaa', 'app', 'pppaaabab', 'ppoppp', 'ob', 'ppaaa', 'b', 'o', 'ppapp', 'oboobbooboooo', 'bbobb']


And, as the next cell exemplifies, the **MTSL grammar** also successfully captured the double harmony pattern. Intuitively, MTSL grammar learned that on the tier of vowels, "o" and "a" should never be adjacent, and on the tier of consonants, restrictions "pb" and "bp" need to be imposed.

In [16]:
print(mtsl_h.generate_sample(25, repeat=False))

['', 'oo', 'ab', 'ooboooob', 'pp', 'bb', 'a', 'bo', 'poo', 'ooop', 'op', 'oopopopoooooopppp', 'oppo', 'apaaaa', 'po', 'aaab', 'obo', 'ppppoopp', 'b', 'o', 'ba', 'paaa', 'bobbooobb', 'aapaaaa', 'ppappaapa']


Overall, we can see that two grammars (SP and MTSL) successfully handled the double harmonic pattern.

  * **SP solution:** within the same word, there cannot be two different vowels and two different consonants;
  * **MTSL solution:** if we only look at vowels, all vowels that are adjacent to each other need to be the same, and similarly for consonants.

### Grammar learning exercise: liquid dissimilation

In several languages (Latin, Georgian, a.o.), liquids tend to alternate. 
Consider the following Latin data: if the final liquid of the stem is "l", the adjectival affix is realized as "aris". And vice versa, if the final liquid is "r", the choice of the affix is "alis".

  * mi<b>l</b>ita<b>r</b>is \~ <*>mi<b>l</b>ita<b>l</b>is _"military"_
  * f<b>l</b>o<b>r</b>a<b>l</b>is \~ <*>f<b>l</b>o<b>r</b>a<b>r</b>is _"floral"_
  * p<b>l</b>u<b>r</b>a<b>l</b>is \~ <*>p<b>l</b>u<b>r</b>a<b>r</b>is _"plural"_
  
We can simplify this pattern by mapping the liquids to themselves, and any intervening vowel or consonant to `c`:

    Good strings: clcccrccl, ccrccclcccrclc, lccccrlcccrcclccc, ...
    Bad strings:  <*>ccrcccrccl, <*>clccrcr, <*>lccrccrc, ...
    Generalization: the liquid following "r" needs to be "l", and vice versa;
                    "c" is irrelevant for the ordering of the liquids. 
                    
The following dataset exemplifies the pattern of liquid dissimilation.

In [17]:
liquid_data = ["", "ccc", "lccrcccclcr", "lrl", "rcclc"]

Use the cell below to explore which language type generalizes the liquid dissimilation pattern the best.

In [18]:
# MEMO:
# initialize the class and provide the data;
# extract the alphabet from the data;
# learn the grammar, possibly switch its polarity;
# generate a sample and see if it complies with the generalization.

# your code here

**NB:** an improved version of the MTSL learning algorithm is coming soon!

## Mappings, transducers and processes

Processes converting one representation to another, or, in other words, rewriting strings, can be encoded via another finite-state device -- **transducer**.
For example, let us encode the following pattern from [de la Higuera (2014)](https://www.cambridge.org/core/books/grammatical-inference/CEEB229AC5A80DFC6436D860AC79434F): _the word-final "a" is rewritten as "1", other "a" are "0". All "b" are translated to "1"._
See the following pairs, for example.

In [19]:
S = [("b", "1"), ("a", "1"), ("ab", "01"), ("abb", "011"), ("bb", "11"), ("aa", "01"), 
     ("aaa", "001"), ("aabaab", "001001"), ("aab", "001"), ("aaba", "0011"), ("aabaa", "00101")]

This generalization can be captured using the following finite state transducer (FST).

<img src="images/higtrans.png" width="290">

There are two **states** of this FST: _"e"_ and "a", where _"e"_ is the unique initial state. The transition $q_e\xrightarrow{\text{b:1}}q_e$ reads "b" in the input, writes "1" to the output, and does not move the FST anywhere from the initial state. $q_e\xrightarrow{\text{a:\emph{e}}}q_a$ reads an "a", moves the machine to another state, but does not write anything to the translation: it will depend on the next symbol.

  * if the next symbol is "a", then it will write "0" to the translation: the previous "a" wasn't word-final;
  * if the next symbol is "b", it writes "01", where "0" is obtained from the non-final "a", and "1" is the translation of "b";
  * if there is no next symbol, the **state output function** defined by $e:e$ and $a:1$ writes "1" to the translation: in this case, "a" occurred in a word-final position.
  
The **input alphabet** of this machine is "a" and "b", and the **output alphabet** is "0" and "1".

The FST module is only designed to work with transducers that read input strings symbol-by-symbol, or, in other words, **subsequential** transducers.

### Encoding an FST

The class `FST` represents finite state transducers, and has the following functionality.

In [20]:
help(FST)

Help on class FST in module fst_object:

class FST(builtins.object)
 |  A class representing finite state transducers.
 |  Attributes:
 |      Q (list): a list of states;
 |      Sigma (list): a list of symbols of the input alphabet;
 |      Gamma (list): a list of symbols of the output alphabet;
 |      qe (str): name of the unique initial state;
 |      E (list): a list of transitions;
 |      stout (dict): a collection of state outputs.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, Sigma=None, Gamma=None)
 |      Initializes the FST object.
 |  
 |  copy_fst(self)
 |      Produces a deep copy of the current FST.
 |      Returns:
 |          T (FST): a copy of the current FST.
 |  
 |  rewrite(self, w)
 |      Rewrites the given string with respect to the rules represented
 |      in the current FST.
 |      Arguments:
 |          w (str): a string that needs to be rewritten.
 |      Outputs:
 |          str: the translation of the input string.
 |  
 |  --------------------

We can then initialize the `FST` class as follows.

In [21]:
fst = FST()
fst.Q = ["", "a"]
fst.Sigma = ["a", "b"]
fst.Gamma = ["0", "1"]
fst.qe = ""
fst.E = [['', 'b', '1', ''], ['', 'a', '', 'a'], ['a', 'b', '01', ''], ['a', 'a', '0', 'a']]
fst.stout = {'': '', 'a': '1'}

### Rewriting string using FSTs

First, we can prepare several examples of strings that can be rewritten using the current FST.

In [22]:
inputs = SL(alphabet=["a", "b"], grammar = [(">", "<")], polar="n").generate_sample(10, repeat=False)
print("Generated strings:", inputs)

Generated strings: ['baab', 'aaa', 'bbb', 'babbba', 'ba', 'baa', 'ab', 'bb', 'a', 'bbbaba']


Then, the method `rewrite` takes a string as input and returns its translation. 

In [23]:
for string in inputs:
    print(string, "--->", fst.rewrite(string))

baab ---> 1001
aaa ---> 001
bbb ---> 111
babbba ---> 101111
ba ---> 11
baa ---> 101
ab ---> 01
bb ---> 11
a ---> 1
bbbaba ---> 111011


### Vowel harmony and consonant harmony, revisited

Earlier in the notebook, we discussed a toy double harmonic pattern where a vowel within a word can be either "o" or "a", and the consonant can be either "b" or "p". 
The well-formed string were of the following type:
    
    ["baabb", "bbboo", "ppaaa", "ppppp"]
    
Now, let us capture the feature changing process instead of defining the well-formedness conditions. Assuming that the spreading moves left to right, we can then mask all non-initial mentions of vowels and consonants inthe words.
    
    [("baABB", "baabb"), ("bBBoA", "bbboo"), ("pBaAA", "ppaaa"), ("pBBBB", "ppppp")]
    
First of all, let us start by defining toy harmonic classes.

In [24]:
specifications = {("a", "o"):"A", ("b", "p"):"B"}

Now, let us generate the training sample.

In [25]:
num_examples = 10
len_examples = 5
S = generate_pairs(num_examples, len_examples, specifications)

show = 5
print(show, "first pairs of S:\n", S[:show])

5 first pairs of S:
 [('abBAB', 'abbab'), ('oAAAb', 'oooob'), ('pBoAB', 'ppoop'), ('poBAB', 'popop'), ('bBoAA', 'bbooo')]


### Learning with OSTIA

Transducers can be extracted from the input and output pairs automatically.
There are several algorithms that rely on different properties and assumptions.
Among the most widely used and discussed learners, there are:

  * **OSTIA** (Onward Subsequential Transducer Inference Algorithm) by [Oncina, Garcia and Vidal (1993)](https://pdfs.semanticscholar.org/9058/01c8e75daacb27d70ccc3c0b587411b6d213.pdf) and [de la Higuera (2014)](https://www.cambridge.org/core/books/grammatical-inference/CEEB229AC5A80DFC6436D860AC79434F);
  * **ISLFLA** (Input Strictly Local Function Learning Algorithm) by [Chandlee, Eyraud and Heinz (2014)](https://hal.archives-ouvertes.fr/hal-01193047/document);
  * **OSLFIA** (Output Strictly Local Function Inference Algorithm) by [Chandlee, Eyraud and Heinz (2015)](https://www.aclweb.org/anthology/W15-2310.pdf), and others.
  
#### Setting up the learner
  
The following section discusses OSTIA learning algorithm, while some of the other ones will be added to _SigmaPie_ codebase soon.

In [26]:
help(ostia)

Help on function ostia in module ostia:

ostia(S, Sigma, Gamma)
    This function implements OSTIA (Onward Subsequential Transduction
    Inference Algorithm).
    Arguments:
        S (list): a list of pairs (o, t), where `o` is the original
            string, and `t` is its translation;
        Sigma (list): the input alphabet;
        Gamma (list): the output alphabet.
    Returns:
        FST: a transducer defining the mapping.



First, let us provide the necessary input to OSTIA and save the resulting machine.

In [27]:
S = generate_pairs(1500, 5, specifications)
Sigma = ["a", "o", "A", "b", "p", "B"]
Gamma = ["a", "o", "b", "p"]
T = ostia(S, Sigma, Gamma)

For a step-by-step implementation of OSTIA, click [here](https://github.com/alenaks/OSTIA/blob/master/ostia.ipynb).

#### Evaluating the performance

In order to evaluate the performance of the obtained automaton, we can generate more input forms.

In [28]:
test = mask_words(generate_words(15, 5, specifications), specifications)
for w in test:
    print(w, "--->", T.rewrite(w))

bBoAA ---> bbooo
apBBB ---> apppp
apAAB ---> apaap
oAAAA ---> ooooo
pBaBA ---> ppapa
bBBaB ---> bbbab
obAAB ---> oboob
oApAA ---> oopoo
apABA ---> apapa
poBBA ---> poppo
apBAB ---> appap
aApBA ---> aappa
oAAbA ---> ooobo
aAApB ---> aaapp
baBBB ---> babbb


As one can see, the performance of OSTIA largely depends on the size of the training sample and the length of the words. 
For example, if the length of words is set to $5$, we need to observe at least $200$ examples in order to see the stably correct outputs.

### Interpretability

After the transducer was extracted, it is possible to explore its structure by simply viweing the list of transitions, states, and state outputs of that machine.

In [29]:
print("States:", T.Q)
print("State outputs:", T.stout)
print("\nTransitions:", T.E)

States: ['', 'b', 'ba', 'a']
State outputs: {'': '', 'b': '', 'ba': '', 'a': ''}

Transitions: [['', 'p', 'p', ''], ['', 'o', 'o', ''], ['', 'b', 'b', 'b'], ['b', 'B', 'b', 'b'], ['b', 'a', 'a', 'ba'], ['ba', 'B', 'b', 'ba'], ['', 'a', 'a', 'a'], ['a', 'b', 'b', 'ba'], ['b', 'o', 'o', 'b'], ['a', 'A', 'a', 'a'], ['ba', 'A', 'a', 'ba'], ['a', 'p', 'p', 'a'], ['a', 'B', 'p', 'a'], ['', 'B', 'p', ''], ['', 'A', 'o', ''], ['b', 'A', 'o', 'b']]


For example, one of the transducers that was build based on the harmonic pairs had the following list of states and transitions. (State outputs are not important in this case: all of them output $\epsilon$.)

    T.Q = ['', 'o', 'b', 'ob']

    T.E = [['', 'p', 'p', ''], ['', 'a', 'a', ''], ['', 'o', 'o', 'o'],
           ['o', 'p', 'p', 'o'], ['', 'b', 'b', 'b'], ['b', 'a', 'a', 'b'],
           ['o', 'A', 'o', 'o'], ['o', 'b', 'b', 'ob'], ['ob', 'A', 'o', 'ob'],
           ['b', 'B', 'b', 'b'], ['b', 'o', 'o', 'ob'], ['ob', 'B', 'b', 'ob'],
           ['o', 'B', 'p', 'o'], ['', 'B', 'p', ''], ['', 'A', 'a', ''],
           ['b', 'A', 'a', 'b']]

We can then visualize this FST, showing that the results of transduction learning are interpretable.

<img src="images/FST.png" width="500">

## Summary

_SigmaPie_ is a toolkit for subregular and subsequential grammar induction.
**Subregular languages and grammar** capture the well-formedness conditions, and **subsequential transducers** are capable of encoding processes, or mappings.

I exemplified the performance of the implemented learning algorithms based on the toy assimilation and dissimilation patterns.
After the FST corresponding to a training sample is built, it is possible to look at the shape of the learned machine, therefore making this approach to grammar construction **fully interpretable**.

**Suggestions** 

If your research can benefit in any way from the extension of _SigmaPie,_ please let me know by shooting me an email at <alena.aksenova@stonybrook.edu>!


**Acknowledgments** 

I am very grateful to [_Thomas Graf_](https://thomasgraf.net/), [_Jeffrey Heinz_](http://jeffreyheinz.net/), [_Aniello De Santo_](https://aniellodesanto.github.io/about/) and _Ayla Karakaş_ whose input on different parts of this project was extremely helpful.